In [1]:
import sys, os, inspect, importlib
import pandas as pd
import geopandas as gpd

from gbdxtools import Interface
from gbdxtools import CatalogImage
from shapely.geometry import box
from shapely.wkt import loads

cmd_folder = os.path.dirname(os.getcwd())
if cmd_folder not in sys.path:
    sys.path.insert(0, cmd_folder)

from GOST_GBDx_Tools import gbdxTasks
from GOST_GBDx_Tools import gbdxURL_misc
from GOST_GBDx_Tools import imagery_search

#In order for the interface to be properly authenticated, follow instructions here:
#   http://gbdxtools.readthedocs.io/en/latest/user_guide.html
#   For Ben, the .gbdx-config file belongs in C:\Users\WB411133 (CAUSE no one else qill f%*$&ing tell you that)
gbdx = Interface()
gbdx.s3.info
curTasks = gbdxTasks.GOSTTasks(gbdx)
gbdxUrl = gbdxURL_misc.gbdxURL(gbdx)


In [9]:
catIDs = ["1040010046439700","1040010032832500","1040010047561800","10400100183B2900","104001002DA46D00"]
for cID in catIDs:
    try:
        xx = CatalogImage(cID)
        xx.metadata
    except Exception as e:
        print(e)

Could not find a catalog entry for the given id: 1040010046439700
Could not find a catalog entry for the given id: 1040010032832500
Could not find a catalog entry for the given id: 1040010047561800
Could not find a catalog entry for the given id: 10400100183B2900
Could not find a catalog entry for the given id: 104001002DA46D00


In [29]:
inputFile = r"C:/Work/Code/catIDs.txt"

catIDs = []
with open(inputFile, 'r') as inFile:
    for line in inFile:
        try:
            xx = int(line[:5])
            catIDs.append(line.replace("\n", ""))
        except:
            pass   

In [32]:
def generateInfo(catID):
    """return array with important information from catID"""
    img = CatalogImage(catID)
    retVals = [catID, loads(img.metadata['image']['imageBoundsWGS84'])]
    for val in ['sensorPlatformName', "acquisitionDate",'satAzimuth','satElevation','cloudCover']:
        retVals.append(img.metadata['image'][val])    
    return(retVals)

allVals = []
for cID in catIDs:
    allVals.append(generateInfo(str(cID)))

C:\WBG\Anaconda\envs\gbdxtools\lib\site-packages\gbdxtools\images\worldview.py:68: UserWarning: Unknown dtype
  warnings.warn('Unknown dtype')


In [34]:
allRes = pd.DataFrame(allVals, columns=['catID','geometry','sensorPlatformName', "acquisitionDate",'satAzimuth','satElevation','cloudCover'])
#allRes_G = gpd.GeoDataFrame(allVals, geometry='geometry')

In [36]:
allRes.to_csv(inputFile.replace(".txt","_MAPPED.csv"))

